In [1]:
# import time
# import math
# import random
import torch
import importlib

from pathlib import Path
from tqdm import tqdm

import os
from pathlib import Path

PROJECT_FOLDER = Path(os.getenv("PROJECT_ROOT"))
DATA_FOLDER = PROJECT_FOLDER / "data"
src_path = PROJECT_FOLDER / "src"

import sys
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    
    
TILES_ROOT = DATA_FOLDER / "tiles"

print("PROJECT_ROOT =", PROJECT_FOLDER)
print("TILES_ROOT =", TILES_ROOT)

PROJECT_ROOT = D:\Divas\Projects\MSCS\7643\Project\TORTOISE
TILES_ROOT = D:\Divas\Projects\MSCS\7643\Project\TORTOISE\data\tiles


In [2]:
from tortoise.normalizer import DataNormalizer
dn = DataNormalizer(DATA_FOLDER / "normalization_stats.json")  # adjust filename if needed

In [ ]:
importlib.reload(importlib.import_module("tortoise.dataset"))
importlib.reload(importlib.import_module("tortoise.dataloader"))

ModuleNotFoundError: No module named 'augmentations'

In [4]:
from tortoise.dataloader import build_dataloaders

train_loader, val_loader, test_loader = build_dataloaders(
    data_root=TILES_ROOT,
    batch_size=16,        # smaller batch for notebook
    normalizer=dn,
    use_rgb=False,
    num_workers=0,
    train_ratio=0.1,
    val_ratio=0.05,
)

print("Train size:", len(train_loader.dataset))
print("Val size:", len(val_loader.dataset))
print("Test size:", len(test_loader.dataset))


Train size: 5847
Val size: 2923
Test size: 49703


In [5]:
from tortoise.train import train_one_epoch, evaluate, masked_bce_loss

In [6]:
import torch
import torch.optim as optim
from tortoise.model import U_Net, init_weights

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = U_Net(img_ch=13, output_ch=1).to(device)
init_weights(model, "kaiming")

optimizer = optim.Adam(model.parameters(), lr=1e-4)

Using device: cpu
initialize network with kaiming


In [ ]:
num_epochs = 5
best_val = float("inf")

for epoch in tqdm(range(1, num_epochs + 1)):
    print(f"\nEpoch {epoch}/{num_epochs}")

    train_loss = train_one_epoch(model, train_loader, optimizer, device=device)
    val_loss   = evaluate(model, val_loader, desc="Val", device=device)

    print(f"Train Loss: {train_loss:.4f}  |  Val Loss: {val_loss:.4f}")

    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), PROJECT_FOLDER /"checkpoints" /"best_unet_5epoch.pth")
        print("  Saved best model")


  0%|          | 0/5 [00:00<?, ?it/s]


Epoch 1/5


  0%|          | 0/5 [08:56<?, ?it/s]


KeyboardInterrupt: 

In [7]:
train_loss = train_one_epoch(model, train_loader, optimizer, device=device)

KeyboardInterrupt: 